In [53]:
import pandas as pd 
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor , AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor


In [2]:
data = pd.read_csv('match_to_match_data.csv')

In [3]:
data.head()

,Unnamed: 0,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,...,result,result_margin,eliminator,method,umpire1,umpire2,season,first_inning_runs,second_inning_runs,total_runs
0,0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,...,runs,140.0,N,Regular,Asad Rauf,RE Koertzen,2008,222,82,304
1,1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,...,runs,33.0,N,Regular,MR Benson,SL Shastri,2008,240,207,447
2,2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,...,wickets,9.0,N,Regular,Aleem Dar,GA Pratapkumar,2008,129,132,261
3,3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,...,wickets,5.0,N,Regular,SJ Davis,DJ Harper,2008,165,166,331
4,4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,...,wickets,5.0,N,Regular,BF Bowden,K Hariharan,2008,110,112,222


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 812 entries, 0 to 811
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          812 non-null    int64  
 1   id                  812 non-null    int64  
 2   city                812 non-null    object 
 3   date                812 non-null    object 
 4   player_of_match     812 non-null    object 
 5   venue               812 non-null    object 
 6   neutral_venue       812 non-null    int64  
 7   team1               812 non-null    object 
 8   team2               812 non-null    object 
 9   toss_winner         812 non-null    object 
 10  toss_decision       812 non-null    object 
 11  winner              812 non-null    object 
 12  result              812 non-null    object 
 13  result_margin       812 non-null    float64
 14  eliminator          812 non-null    object 
 15  method              812 non-null    object 
 16  umpire1 

In [5]:
data.describe()

,Unnamed: 0,id,neutral_venue,result_margin,season,first_inning_runs,second_inning_runs,total_runs
count,812.000000,8.120000e+02,812.000000,812.000000,812.000000,812.000000,812.000000,812.000000
mean,407.066502,7.559630e+05,0.094828,17.044335,2013.921182,162.024631,148.623153,310.647783
std,235.871181,3.061363e+05,0.293157,21.998666,3.667929,29.982640,30.114551,54.373772
min,0.000000,3.359820e+05,0.000000,0.000000,2008.000000,67.000000,44.000000,135.000000
25%,202.750000,5.012268e+05,0.000000,6.000000,2011.000000,143.000000,132.000000,279.000000
50%,406.500000,7.292960e+05,0.000000,8.000000,2014.000000,163.000000,150.000000,314.000000
75%,611.250000,1.082626e+06,0.000000,19.000000,2017.000000,182.000000,169.000000,345.000000
max,815.000000,1.237181e+06,1.000000,146.000000,2020.000000,263.000000,226.000000,469.000000


In [6]:
data.columns

Index(['Unnamed: 0', 'id', 'city', 'date', 'player_of_match', 'venue',
       'neutral_venue', 'team1', 'team2', 'toss_winner', 'toss_decision',
       'winner', 'result', 'result_margin', 'eliminator', 'method', 'umpire1',
       'umpire2', 'season', 'first_inning_runs', 'second_inning_runs',
       'total_runs'],
      dtype='object')

In [7]:
data['avg_score'] = (data['first_inning_runs'] + data['second_inning_runs']) // 2

In [8]:
data.head()

,Unnamed: 0,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,...,result_margin,eliminator,method,umpire1,umpire2,season,first_inning_runs,second_inning_runs,total_runs,avg_score
0,0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,...,140.0,N,Regular,Asad Rauf,RE Koertzen,2008,222,82,304,152
1,1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,...,33.0,N,Regular,MR Benson,SL Shastri,2008,240,207,447,223
2,2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,...,9.0,N,Regular,Aleem Dar,GA Pratapkumar,2008,129,132,261,130
3,3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,...,5.0,N,Regular,SJ Davis,DJ Harper,2008,165,166,331,165
4,4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,...,5.0,N,Regular,BF Bowden,K Hariharan,2008,110,112,222,111


In [9]:
 df= data.drop(columns=['id' , 'date' , 'player_of_match' , 'neutral_venue' , 'toss_winner', 'toss_decision',
       'winner', 'result', 'result_margin', 'method', 'umpire1',
       'umpire2', 'season' , 'Unnamed: 0'])

In [18]:
df

,city,venue,team1,team2,eliminator,first_inning_runs,second_inning_runs,total_runs,avg_score
0,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,Kolkata Knight Riders,N,222,82,304,152
1,Chandigarh,"Punjab Cricket Association Stadium, Mohali",Kings XI Punjab,Chennai Super Kings,N,240,207,447,223
2,Delhi,Feroz Shah Kotla,Delhi Daredevils,Rajasthan Royals,N,129,132,261,130
3,Mumbai,Wankhede Stadium,Mumbai Indians,Royal Challengers Bangalore,N,165,166,331,165
4,Kolkata,Eden Gardens,Kolkata Knight Riders,Deccan Chargers,N,110,112,222,111
...,...,...,...,...,...,...,...,...,...
807,Dubai,Dubai International Cricket Stadium,Royal Challengers Bangalore,Mumbai Indians,Y,201,201,402,201
808,Dubai,Dubai International Cricket Stadium,Mumbai Indians,Delhi Capitals,N,200,143,343,171
809,Abu Dhabi,Sheikh Zayed Stadium,Royal Challengers Bangalore,Sunrisers Hyderabad,N,131,132,263,131
810,Abu Dhabi,Sheikh Zayed Stadium,Delhi Capitals,Sunrisers Hyderabad,N,189,172,361,180


In [20]:
# Preparing X and y variables
X = df.drop(columns=['avg_score'])

In [21]:
y = df['avg_score']

In [26]:
# seprating numerical and categorical features
cat_feat = X.select_dtypes(include='object').columns
num_feat = X.select_dtypes(exclude='object').columns

In [34]:
print('number of unique values in City ', X['city'].nunique())
print('number of unique values in venue ', X['venue'].nunique())
print('number of unique values in team1 ', X['team1'].nunique())
print('number of unique values in team2 ', X['team2'].nunique())

number of unique values in City  32
number of unique values in venue  36
number of unique values in team1  14
number of unique values in team2  14


In [49]:
# Column Transformer

from sklearn.preprocessing import OneHotEncoder , LabelEncoder  , StandardScaler
from sklearn.compose import ColumnTransformer

oh_enc = OneHotEncoder()
label_enc  =LabelEncoder()
std_enc = StandardScaler()

preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder' , oh_enc , cat_feat),
#         ('LabelEncoder' , label_enc , ['eliminator']),                       label encoder doesn't work in pipeline
        ('StandardScaler' , std_enc , num_feat)
    ]
)
 


In [50]:
X = preprocessor.fit_transform(X)

In [51]:
X.shape

(812, 101)

In [52]:
# train test split
from sklearn.model_selection import train_test_split

X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.2 , random_state=9)


In [59]:
# evaluating the model

from sklearn.metrics import r2_score , mean_absolute_error , mean_squared_error

def evaluate_mode(true , pred):
    mse = mean_squared_error(true , pred)
    r2 = r2_score(true , pred)
    mae = mean_absolute_error(true , pred)
    
    return mse , r2 , mae

In [64]:
# model training

models = {
    'LinearRegression': LinearRegression(),
    'RandomForestRegressor' : RandomForestRegressor(),
    'KNeighborsRegressor': KNeighborsRegressor(n_neighbors=5),
    'DecisionTree' : DecisionTreeRegressor(),
    'AdaBoost': AdaBoostRegressor(),
    'CatBoost':CatBoostRegressor(verbose=False),
    'XGBRegressor':XGBRegressor()
}

In [67]:
for i in range(len(models)):
    model = list(models.values())[i]
    model.fit(X_train , y_train)
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    train_mse , train_r2 , train_mae = evaluate_mode(y_train , y_train_pred)
    test_mse , test_r2 , test_mae = evaluate_mode(y_test , y_test_pred)  
    
    print(list(models.keys())[i])
    print('*' *32)
    
    print('-'*32)    
    print('Train Data Score')
    print('-'*32)    
    print("Mean Squared Error:" , train_mse)
    print("Mean Absolute Error:" , train_mae)
    print("r2 Score:" , train_r2)
    print('-'*32)    
    
    print('Test Data Score')
    print('-' *32)
    print("Mean Squared Error:" , test_mse)
    print("Mean Absolute Error:" , test_mae)
    print("r2 Score:" , test_r2)
    print('-'*32)    

    print('\n')
    
    
    
    

LinearRegression
********************************
--------------------------------
Train Data Score
--------------------------------
Mean Squared Error: 0.05525011239379016
Mean Absolute Error: 0.22206144561215652
r2 Score: 0.9999224130066665
--------------------------------
Test Data Score
--------------------------------
Mean Squared Error: 0.06769820503535802
Mean Absolute Error: 0.2419944650072986
r2 Score: 0.9999197739878901
--------------------------------


RandomForestRegressor
********************************
--------------------------------
Train Data Score
--------------------------------
Mean Squared Error: 0.07486533127889052
Mean Absolute Error: 0.07473035439137064
r2 Score: 0.9998948676173283
--------------------------------
Test Data Score
--------------------------------
Mean Squared Error: 1.5404276073619618
Mean Absolute Error: 0.336748466257668
r2 Score: 0.9981745104789992
--------------------------------


KNeighborsRegressor
********************************
------